<a href="https://colab.research.google.com/github/yh05/kadai01/blob/master/kaggle_first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
files.upload()



KeyboardInterrupt: ignored

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [7]:
!pip install kaggle

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
!kaggle competitions download -c digit-recognizer

 76% 56.0M/73.2M [00:00<00:00, 149MB/s]
100% 73.2M/73.2M [00:00<00:00, 162MB/s]
 66% 32.0M/48.8M [00:00<00:00, 40.3MB/s]
100% 48.8M/48.8M [00:00<00:00, 88.6MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 34.5MB/s]


In [0]:
!pip install -q keras

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau  
from sklearn.model_selection import train_test_split


In [0]:
# load training & test datasets
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [0]:
# pandas to numpy
y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)

del train

# normalize
X_train = X_train/255.0
test = test/255.0

# reshape the data so that the data
# represents (label, img_rows, img_cols, grayscale)
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# one-hot vector as a label
y_train = to_categorical(y_train, num_classes=10)

In [0]:
# convolution -> batch normalization -> ReLU actuvation -> pooling
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

In [0]:
# compile model
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [0]:
# cross validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=1220)

In [0]:
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
train_generator = gen.flow(X_train, y_train, batch_size=64)

In [0]:
# learning rate
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)


In [32]:
# model training
model.fit_generator(train_generator, epochs=30, validation_data = (X_val, y_val), verbose=2, steps_per_epoch=X_train.shape[0]/36,
                                       callbacks=[learning_rate_reduction])

Epoch 1/30
 - 326s - loss: 0.0695 - acc: 0.9784 - val_loss: 0.0560 - val_acc: 0.9850
Epoch 2/30
 - 334s - loss: 0.0457 - acc: 0.9856 - val_loss: 0.0620 - val_acc: 0.9871
Epoch 3/30
 - 328s - loss: 0.0350 - acc: 0.9894 - val_loss: 0.0430 - val_acc: 0.9886
Epoch 4/30
 - 339s - loss: 0.0339 - acc: 0.9904 - val_loss: 0.0339 - val_acc: 0.9910
Epoch 5/30
 - 350s - loss: 0.0299 - acc: 0.9910 - val_loss: 0.0455 - val_acc: 0.9886
Epoch 6/30
 - 344s - loss: 0.0270 - acc: 0.9919 - val_loss: 0.0634 - val_acc: 0.9876
Epoch 7/30
 - 341s - loss: 0.0269 - acc: 0.9925 - val_loss: 0.0423 - val_acc: 0.9914
Epoch 8/30
 - 339s - loss: 0.0237 - acc: 0.9931 - val_loss: 0.0481 - val_acc: 0.9881
Epoch 9/30
 - 343s - loss: 0.0215 - acc: 0.9937 - val_loss: 0.0569 - val_acc: 0.9910
Epoch 10/30
 - 345s - loss: 0.0221 - acc: 0.9937 - val_loss: 0.0480 - val_acc: 0.9910

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 11/30
 - 345s - loss: 0.0136 - acc: 0.9961 - val_loss: 0.0366 

In [0]:
# model prediction on test data
predictions = model.predict_classes(test, verbose=0)

In [0]:
# make a submission file
submissions = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
    "Label": predictions})
submissions.to_csv("my_submission.csv", index=False, header=True)

In [0]:
# submit the file to kaggle
!kaggle competitions submit digit-recognizer -f my_submission.csv -m "Yeah! I submit my file through the Google Colab!"